In [1]:
import os
import pickle
import math
from nilearn import connectome
from sklearn import cluster, metrics
import matplotlib.pyplot as plt
import gc

In [2]:
with open("time_series2.pkl", "rb") as f:
    data = pickle.load(f)

In [3]:
min_window_length_Second = 30
max_window_length_Second = 200
window_length_TR = 22
slidingSteps = 1
ks = list(range(2, 25))

In [4]:
def sliceWindows(time_series, frame, interval):
    if len(time_series) < frame:
        return [time_series]
    windows = []
    l, r = 0, frame
    while r < time_series.shape[0]:
        windows.append(time_series[l:r])
        l += interval
        r = l + frame
    return windows

In [5]:
time_series = data["sub-307"]["ses-preop"]["run-01"]["time_series"]
subs_num = 5

In [8]:
def clustering(windows, save_path):
    fcs = connectome.ConnectivityMeasure(kind="correlation").fit_transform(windows)

    inertias = []
    scs = []
    chs = []
    dbs = []
    fcs2d = fcs.reshape((fcs.shape[0], 13456))
    for k in ks:
        if k < fcs2d.shape[0]:
            center, states, inertia = cluster.k_means(fcs2d, k)
            inertias.append(inertia) # 肘点法
            scs.append(metrics.silhouette_score(fcs2d, states)) # 轮廓系数
            chs.append(metrics.calinski_harabasz_score(fcs2d, states)) # CH，方差比
            dbs.append(metrics.davies_bouldin_score(fcs2d, states)) # DB
        else:
            inertias.append(inertias[-1])
            scs.append(scs[-1])
            chs.append(chs[-1])
            dbs.append(dbs[-1])
    # 绘图
    figi, axi = plt.subplots(2, 2, figsize=(20, 10))
    figi.patch.set_color("white")
    axi[0, 0].set_title("elbow method")
    axi[0, 0].plot(ks, inertias)
    axi[0, 1].set_title("Silhouette Coefficient")
    axi[0, 1].plot(ks, scs)
    axi[1, 0].set_title("calinski harabasz")
    axi[1, 0].plot(ks, chs)
    axi[1, 1].set_title("davies bouldin")
    axi[1, 1].plot(ks, dbs)
    figi.savefig(save_path, format="png")
    plt.cla()
    plt.clf()
    plt.close("all")


In [9]:
# preop
save_dir_preop = f"cluster_evaluate/{subs_num}_subs/preop/"
save_dir_postop = f"cluster_evaluate/{subs_num}_subs/postop/"
os.makedirs(save_dir_preop, exist_ok=True)
os.makedirs(save_dir_postop, exist_ok=True)
for time in range(50, 201, 10):
    count = 0
    save_path_preop = f"{save_dir_preop}/{time}s.png"
    save_path_postop = f"{save_dir_postop}/{time}s.png"
    if os.path.exists(save_dir_preop) and os.path.exists(save_path_postop):
        continue
    for sub, ses in data.items():
        if count >= subs_num:
            break
        count += 1
        windows_preop = []
        windows_postop = []
        for run, items in ses["ses-preop"].items():
            window_length_TR = math.ceil(time/items["TR"])
            slidingSteps = math.ceil(time/items["TR"])
            windows_preop += sliceWindows(items["time_series"], window_length_TR, slidingSteps)
        for run, items in ses["ses-postop"].items():
            window_length_TR = math.ceil(time/items["TR"])
            slidingSteps = math.ceil(time/items["TR"])
            windows_postop += sliceWindows(items["time_series"], window_length_TR, slidingSteps)
    # preop
    if len(windows_preop) > 2:
        if not os.path.exists(save_path_preop):
            clustering(windows_preop, save_path_preop)
    # postop
    if len(windows_postop) > 2:
        if not os.path.exists(save_path_postop):
            clustering(windows_postop, save_path_postop)
    plt.cla()
    plt.clf()
    plt.close("all")
    gc.collect()